In [50]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import time
import re
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm_notebook

In [51]:
df = pd.read_csv('./Reviews_csv/Reviews_all_clean.csv', index_col = 0)

In [65]:
idlist = df['Id'].unique().tolist()

In [66]:
df.head()

,Id,Rating,Review
0,2255550301,불만,배송도 너무 늦고.상품은 실밥 처리가 제대로 안돼있어요
1,2255550301,불만,입고 밖에 나갔다왔는데 옆구리가 뜯어져았었더라구요 ; 버려야죠 뭐
2,2255550301,불만,가슴부분에 작은구멍이 나있어요.. 목부분은 제대로 봉합안되서 왼쪽오른쪽 짝짝이네요..
3,2255550301,불만,그레이주문했는데 아이보리가 왔네요 반품하려다 귀찮아서 그냥 입으려고요
4,2255550301,불만,팔목쪽이터져잇어요 반품기다리는 시간귀찮아서 입어요


In [67]:
cnt_reviews = df.pivot_table(index = 'Id', values = 'Review', aggfunc= len)

In [68]:
atleast5 = cnt_reviews[cnt_reviews['Review'] > 4]

In [69]:
idlist = atleast5.index.tolist()

In [70]:
len(idlist)

617

In [84]:
urllist

['/itemdetail/2255550301?inflow=wsl',
 '/itemdetail/2257173591?inflow=wsl',
 '/itemdetail/2257379077?inflow=wsl',
 '/itemdetail/2257693815?inflow=wsl',
 '/itemdetail/2257840250?inflow=wsl',
 '/itemdetail/2257914434?inflow=wsl',
 '/itemdetail/2258628829?inflow=wsl',
 '/itemdetail/2258653320?inflow=wsl',
 '/itemdetail/2259921237?inflow=wsl',
 '/itemdetail/2259986983?inflow=wsl',
 '/itemdetail/2260559572?inflow=wsl',
 '/itemdetail/2260784797?inflow=wsl',
 '/itemdetail/2260848627?inflow=wsl',
 '/itemdetail/2261436573?inflow=wsl',
 '/itemdetail/2261771783?inflow=wsl',
 '/itemdetail/2262864914?inflow=wsl',
 '/itemdetail/2262882947?inflow=wsl',
 '/itemdetail/2262891897?inflow=wsl',
 '/itemdetail/2263036438?inflow=wsl',
 '/itemdetail/2263129891?inflow=wsl',
 '/itemdetail/2263353984?inflow=wsl',
 '/itemdetail/2263375642?inflow=wsl',
 '/itemdetail/2263945836?inflow=wsl',
 '/itemdetail/2264854206?inflow=wsl',
 '/itemdetail/2265032083?inflow=wsl',
 '/itemdetail/2270574041?inflow=wsl',
 '/itemdetai

In [71]:
urllist = ['/itemdetail/' + str(i) + '?inflow=wsl' for i in idlist]

In [77]:
driver = webdriver.Chrome('/Users/hwii/Downloads/chromedriver_mac64/chromedriver')
root = "https://swindow.naver.com"

In [80]:
def click_item(variable):
    driver.execute_script("return arguments[0].scrollIntoView();", variable)
    time.sleep(0.5)
    ActionChains(driver).click(variable).perform()
    time.sleep(0.5)

In [81]:
reviews = []
ratings = []
error_url = []
product_id = []

for url in tqdm_notebook(urllist[7671:]):
    driver.get(root+url)
    time.sleep(0.5)
    prod_id = re.split('\?|/', url)[2]
    try:
        # 0. 리뷰 가져올 준비
        # 리뷰 섹션 클릭하기
        showreview = driver.find_element_by_xpath("""//*[@id="content"]/div[1]/div[2]/ul/li[2]/a""")
        click_item(showreview)
        
        bad1 = driver.find_element_by_xpath("""//*[@id="premium_grade_list"]/li[5]/a""")
        click_item(bad1)
        
        showmore = driver.find_elements_by_class_name('button_more')
        for button in showmore:
            click_item(button)
            
        bad2 = driver.find_element_by_xpath("""//*[@id="grade_list"]/li[4]/a""")
        click_item(bad2)
       
        # 1. 리뷰 가져오기
        # 1-1. 프리미엄 리뷰 (제목), 일반 리뷰 (본문) 소스 구분 없이 다 찾기
        rev = driver.find_elements_by_class_name("header_review_comment")
        rev_total_cnt = len(rev) # 전체 리뷰 갯수 저장
        
        # 1-2. 프리미엄 리뷰 본문만 따로 찾기
        prm_rev_location = driver.find_elements_by_class_name("area_full_article");
        prm_cnt = len(prm_rev_location) # 프리미엄 리뷰 갯수 저장
        
        # 1-3. 1-1에서 찾은 프리미엄 리뷰 제목이랑 본문 합칠건데, 리스트 길이가 안 맞으니까 맞춰주기
        prm_extend = [''] * (len(rev) - len(prm_rev_location))
        prm_rev_contents = [item.text.split('\n판매자 ')[0] for item in prm_rev_location] + prm_extend
        
        # 1-4. 일반 리뷰 소스에서 본문 text 가져오기
        rev_content = [item.text for item in rev]
        
        # 1-5. 프리미엄 : 제목이랑 본문 합치기, 일반 리뷰
        total_content = [rev_content[i] + ' ' + prm_rev_contents[i] for i in range(rev_total_cnt)]
        
        # 1-6. 리뷰 total_content에 저장!
        reviews.extend(total_content)
        
        # 2. 만족도 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        rate_loc = soup.find_all('div', 'col_label') # 만족도 소스 찾기
        rate = [r.get_text().strip() for r in rate_loc] # 텍스트만 따로 저장
        ratings.extend(rate) # rating 에 저장
        
        # 3. 상품 번호 저장 (리뷰의 수만큼! 같은 상품의 리뷰임을 알아야하니까)
        product_id.extend([prod_id] * rev_total_cnt)

    except:
        # 에러난 애들 ㅠㅠ
        product_id.append(prod_id) # 상품 번호 기록
        reviews.append(None) # 리뷰, 만족도에는 None
        ratings.append(None) 
        error_url.append(url) # 따로 url 저장

In [62]:
len(reviews), len(ratings), len(product_id)

(8347, 8347, 8347)

In [63]:
df = pd.DataFrame({'Review': reviews, 'Rating': ratings, 'Id': product_id})
df.to_csv('Reviews_Bad_15719_24066.csv', encoding = 'utf-8')

thefile = open('error_url_bad_15719_24066.txt', 'w')
for item in error_url:
    thefile.write("%s\n" % item)